In [1]:
import yaml
import glob
from shortid import ShortId
from rdflib import plugin, ConjunctiveGraph, Graph, URIRef, Literal, BNode
from rdflib.store import Store
from rdflib.namespace import RDF, RDFS, XSD

ns = 'http://envri.eu/ns/'
sid = ShortId()
store = plugin.get('IOMemory', Store)()

vocab = dict()
vocab['ResearchInfrastructure'] = URIRef('http://www.oil-e.net/ontology/envri-rm.owl#ResearchInfrastructure')
vocab['date'] = URIRef('http://purl.org/dc/terms/date')
vocab['creator'] = URIRef('http://purl.org/dc/terms/creator')
vocab['hasVersion'] = URIRef('http://purl.org/dc/terms/hasVersion')
vocab['name'] = URIRef('http://xmlns.com/foaf/0.1/name')
vocab['mbox'] = URIRef('http://xmlns.com/foaf/0.1/mbox')

def process(document):
    n1 = BNode()
    n2 = BNode()
    gid = URIRef('{}G{}'.format(ns, sid.generate()))
    g = Graph(store, gid)
    g.add((gid, vocab['date'], Literal(document['survey']['date'], datatype=XSD.dateTime)))
    g.add((gid, vocab['hasVersion'], Literal(document['survey']['version'])))
    g.add((gid, vocab['creator'], n1))
    g.add((n1, vocab['name'], Literal(document['survey']['creator']['name'])))
    g.add((n1, vocab['mbox'], Literal(document['survey']['creator']['email'])))
    
    g.add((n2, RDF.type, vocab['ResearchInfrastructure']))
    g.add((n2, RDFS.label, Literal(document['infrastructure']['label'])))
    

for file in glob.glob('descriptions/*.yaml'):
    with open(file, 'r') as f:
        for document in yaml.load_all(f, Loader=yaml.FullLoader):
            process(document)
    
g = ConjunctiveGraph(store)
g.bind('envri', ns)
g.bind('dcterms', 'http://purl.org/dc/terms/')
g.bind('foaf', 'http://xmlns.com/foaf/0.1/')
g.serialize(destination='data.trig', format='trig')

ScannerError: mapping values are not allowed here
  in "descriptions/elter-20190319.yaml", line 13, column 15

In [75]:
import io
import pandas as pd
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer

q = """
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?date ?infrastructure WHERE {
    ?g dcterms:date ?date .
    ?g dcterms:hasVersion "latest" .
    GRAPH ?g { 
        ?ri a rm:ResearchInfrastructure .
        ?ri rdfs:label ?infrastructure . 
    }
}
"""

serializer = CSVResultSerializer(g.query(q))
output = io.BytesIO()
serializer.serialize(output)
display(pd.read_csv(io.StringIO(output.getvalue().decode('utf-8'))))

,date,infrastructure
0,2019-03-19,eLTER
